# Registration example using multiscale-imaging
This notebook contains an example of applying the registration with the functions transferred into the multiscale-imaging package.

In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.microscopy.ome as ome
import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt

# Set up registration
First define the paths to each needed file.  This includes the transform parameters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [2]:
# The input files are available at https://uwmadison.box.com/s/05ou19dsi2y2wfzgocf1ab7cbbeb4fgn
fiducial_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\Fiducial')
mouse_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\For registration')

registered_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\Registered')

In [3]:
microscopy_path = Path(mouse_dir, 'SHG down 8x.tif')
microscopy_tile_path = Path(mouse_dir, 'SHG settings.ome.tif')

us_path = Path(mouse_dir, 'L38 Mouse 210 Z13595 75dB.tif')
us_pl_path = Path(mouse_dir, 'L38 Mouse 210 Z13595.pos')
us_param_path = Path(mouse_dir, 'L38 Mouse 210 Z13595_Run-1_Settings.mat')

transform_path = Path(fiducial_dir, 'L38 Transform.txt')

Now open the image.  If the spacing information is not properly written to either image it can still be set manually.

In [4]:
spacing = [12, 12, 12]
gauge_height = 13595
us_image = coord.open_us(us_path, us_pl_path, us_param_path, spacing, gauge_height)

In [5]:
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=8)

Load the coordinate transform

In [6]:
with open(transform_path, 'rb') as fp:
    coordinate_transform = pickle.load(fp)
    print(coordinate_transform)

[-3104.35189186  3773.31892593 20999.71167947]


IBC

In [35]:
analysis_dir = Path(r'F:\Research\LINK\Mouse images\2019-10-21 - Mouse 210v2\L38 Analysis 0-25 latwindow')
ibc_path = Path(analysis_dir, 'L38 Analysis 0-25 latwindow_IBC.tif')
ibc_spacing = [34.5, 100, 12]
dynamic_range = 70
ibc_image = coord.open_us(ibc_path, us_pl_path, us_param_path, ibc_spacing, gauge_height, dynamic_range)

Apply the registration and save the results

In [36]:
ibc_reg_path = Path(registered_dir, 'L38 IBC and SHG on SHG coords 0-25 latwindow.tif')

In [40]:
us_reg = tran.apply_transform_params(microscopy_image, us_image, 
                                     coordinate_transform,
                                     sitk.TranslationTransform(3, [0, 0, 0]))

In [41]:
ibc_reg = tran.apply_transform_params(microscopy_image, ibc_image, 
                                     coordinate_transform,
                                     sitk.TranslationTransform(3, [0, 0, 0]))

In [42]:
ome.save_ijstyle_overlay([us_reg[:, :, 4:20], microscopy_image[:, :, 4:20], ibc_reg[:, :, 4:20]], ibc_reg_path)